# model 2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import numpy as np

# Load and preprocess MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.expand_dims(X_train, -1).astype("float32") / 255.0
X_test = np.expand_dims(X_test, -1).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Define original CNN model
def create_original_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Compile and train original model
original_model = create_original_model()
original_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [ ]:
original_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
original_model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

# Evaluate original model
original_loss, original_accuracy = original_model.evaluate(X_test, y_test)
print(f"Original Model Accuracy: {original_accuracy}, Loss: {original_loss}")

Epoch 1/10
1500/1500 [==============================] - 40s 26ms/step - loss: 0.1654 - accuracy: 0.9501 - val_loss: 0.0703 - val_accuracy: 0.9803
Epoch 2/10
1500/1500 [==============================] - 36s 24ms/step - loss: 0.0520 - accuracy: 0.9840 - val_loss: 0.0498 - val_accuracy: 0.9856
Epoch 3/10
1500/1500 [==============================] - 39s 26ms/step - loss: 0.0368 - accuracy: 0.9881 - val_loss: 0.0378 - val_accuracy: 0.9896
Epoch 4/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.0269 - accuracy: 0.9913 - val_loss: 0.0454 - val_accuracy: 0.9865
Epoch 5/10
1500/1500 [==============================] - 39s 26ms/step - loss: 0.0197 - accuracy: 0.9936 - val_loss: 0.0362 - val_accuracy: 0.9899
Epoch 6/10
1500/1500 [==============================] - 37s 25ms/step - loss: 0.0148 - accuracy: 0.9947 - val_loss: 0.0359 - val_accuracy: 0.9906
Epoch 7/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.0118 - accuracy: 0.9960 - val_loss: 0.0439 -

In [ ]:
import os

# model_save_path = 'cnn_model.h5'
model_save_path = 'model2.keras'
original_model.save(model_save_path)

# Get the size of the model in bytes
model_size = os.path.getsize(model_save_path)
print(f"Model size: {model_size / 1024:.2f} KB")

Model size: 1465.01 KB


# Pruning

In [ ]:
! pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot  # TensorFlow Model Optimization toolkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
import tf_keras as keras

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pruning_params = {
      # 'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=100)
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.1, begin_step=0, frequency=100)
  }

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model = prune_low_magnitude(original_model, **pruning_params)

# Use smaller learning rate for fine-tuning
opt = keras.optimizers.Adam(learning_rate=1e-5)

pruned_model.compile(
  loss=keras.losses.CategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy']
)

pruned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 26, 26, 32)        610       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 32)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_conv2d  (None, 11, 11, 64)        36930     
 _1 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_po  (None, 5, 5, 64)          1         
 oling2d_1 (PruneLowMagnitu                                      
 de)                                                    

In [ ]:
pruned_model.fit(
  X_train,
  y_train,
  epochs=10,
  validation_split=0.2,
  callbacks=callbacks)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tf_keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 [==============================] - 43s 27ms/step - loss: 0.0071 - accuracy: 0.9978 - val_loss: 0.0414 - val_accuracy: 0.9912
Epoch 2/10
1500/1500 [==============================] - 39s 26ms/step - loss: 0.0031 - accuracy: 0.9989 - val_loss: 0.0394 - val_accuracy: 0.9916
Epoch 3/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0388 - val_accuracy: 0.9921
Epoch 4/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0386 - val_accuracy: 0.9924
Epoch 5/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.0384 - val_accuracy: 0.9923
Epoch 6/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0383 - val_accuracy: 0.9923
Epoch 7/10
1500/1500 [==============================] - 38s 26ms/step - loss: 8.8877e-04 - accuracy: 0.9998 - val_loss: 0.0384 - val_ac

In [ ]:
model_save_path = 'cnn_pruned_model.keras'
pruned_model.save(model_save_path)

# Get the size of the model in bytes
model_size = os.path.getsize(model_save_path)
print(f"Model size after pruning: {model_size / 1024:.2f} KB")

Model size after pruning: 2439.63 KB


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
tflite_model = converter.convert()

# Save the TFLite model to a file
model_save_path = 'pruned_model.tflite'
with open('pruned_model.tflite', 'wb') as f:
    f.write(tflite_model)

model_size = os.path.getsize(model_save_path)
print(f"Model size after pruning + convert to tf lite: {model_size / 1024:.2f} KB")

Model size after pruning + convert to tf lite: 479.57 KB


# Quantization

In [ ]:
# Strip the pruning wrappers to finalize the pruned model
final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Convert to TensorFlow Lite model with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)

# Set quantization parameters
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Default optimization includes quantization

# Optionally, if you have representative data, you can enable full integer quantization:
# Provide a representative dataset for better accuracy in quantization
def representative_dataset():
    for data in X_test.take(100):  # use a small sample of your data <<< what if we change this to 1000? to all data? 70rb? how?
        yield [tf.dtypes.cast(data, tf.float32)] # can we change this to integer?

converter.representative_dataset = representative_dataset
converter.target_spec.supported_types = [tf.float16]  # This can be int8 as well, hm... << nope, error

# Convert the model
quantized_tflite_model = converter.convert()

# Save the quantized model to a file
with open('quantized_pruned_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

# To load and use the TFLite model later:
interpreter = tf.lite.Interpreter(model_path='quantized_pruned_model.tflite')
interpreter.allocate_tensors()

In [ ]:
final_model.compile(
  loss=keras.losses.CategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
final_model.fit(
  X_train,
  y_train,
  epochs=10,
  validation_split=0.2,
  callbacks=callbacks)

Epoch 1/10
1500/1500 [==============================] - 35s 23ms/step - loss: 4.5197e-04 - accuracy: 1.0000 - val_loss: 0.0394 - val_accuracy: 0.9927
Epoch 2/10
1500/1500 [==============================] - 38s 26ms/step - loss: 3.8399e-04 - accuracy: 1.0000 - val_loss: 0.0398 - val_accuracy: 0.9926
Epoch 3/10
1500/1500 [==============================] - 37s 25ms/step - loss: 3.3273e-04 - accuracy: 1.0000 - val_loss: 0.0401 - val_accuracy: 0.9924
Epoch 4/10
1500/1500 [==============================] - 39s 26ms/step - loss: 2.8544e-04 - accuracy: 1.0000 - val_loss: 0.0405 - val_accuracy: 0.9923
Epoch 5/10
1500/1500 [==============================] - 34s 23ms/step - loss: 2.4923e-04 - accuracy: 1.0000 - val_loss: 0.0408 - val_accuracy: 0.9923
Epoch 6/10
1500/1500 [==============================] - 37s 25ms/step - loss: 2.1775e-04 - accuracy: 1.0000 - val_loss: 0.0412 - val_accuracy: 0.9922
Epoch 7/10
1500/1500 [==============================] - 36s 24ms/step - loss: 1.8964e-04 - accuracy:

In [ ]:
model_size = os.path.getsize('quantized_pruned_model.tflite')
print(f"Model size after quantization: {model_size / 1024:.2f} KB")

Model size after quantization: 242.38 KB


# Optimized Model with Knowledge Distillation

In [ ]:
# Custom loss function for Knowledge Distillation
def distillation_loss(y_true, y_pred, teacher_logits, temperature=5.0, alpha=0.5):
    student_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    distillation_loss = tf.keras.losses.KLDivergence()(
        tf.nn.softmax(teacher_logits / temperature),
        tf.nn.softmax(y_pred / temperature)
    )
    return alpha * student_loss + (1 - alpha) * distillation_loss

# Train student model using knowledge distillation
# student_model = create_student_model()
# student_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
student_model = original_model

# Train student model with teacher model's logits
def train_student_model(student_model, teacher_model, X_train, y_train):
    teacher_logits = teacher_model.predict(X_train)

    # Custom training loop
    for epoch in range(10):
        print(f"Epoch {epoch + 1}/10")
        for i in range(0, len(X_train), 32):
            X_batch = X_train[i:i+32]
            y_batch = y_train[i:i+32]
            with tf.GradientTape() as tape:
                y_pred = student_model(X_batch, training=True)
                loss = distillation_loss(y_batch, y_pred, teacher_logits[i:i+32])
            grads = tape.gradient(loss, student_model.trainable_variables)
            student_model.optimizer.apply_gradients(zip(grads, student_model.trainable_variables))

# Train the student model
train_student_model(student_model, final_model, X_train, y_train)

1875/1875 [==============================] - 12s 6ms/step
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [ ]:
# Evaluate student model
student_loss, student_accuracy = student_model.evaluate(X_test, y_test)
print(f"Student Model Accuracy: {student_accuracy}, Loss: {student_loss}")

313/313 [==============================] - 3s 9ms/step - loss: 0.0630 - accuracy: 0.9889
Student Model Accuracy: 0.9889000058174133, Loss: 0.0629761591553688


In [ ]:
# model_save_path = 'cnn_model.h5'
model_save_path = 'student_model.keras'
student_model.save(model_save_path)

# Get the size of the model in bytes
model_size = os.path.getsize(model_save_path)
print(f"Model size: {model_size / 1024:.2f} KB")

Model size: 1465.01 KB
